In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd

In [2]:
df_train = pd.read_csv("trac2_CONVT_train.csv")
columnsTrain = df_train["text"]

In [8]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re
import string

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

def clean_text(text):
    text = text.lower()

    text = re.sub(r'[^a-z\s]', '', text)
    
    text = text.translate(str.maketrans('', '', string.punctuation))

    tokens = nltk.word_tokenize(text)

    clean_sentence = ' '.join(tokens)
    return clean_sentence

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tranx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\tranx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tranx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
emotionVocab = set()
listSentence = []
listWordVector = []

maxLengthSentence = 0

for column in columnsTrain:
    cleanColumn = clean_text(column)
    listSentence.append(cleanColumn)

for sentence in listSentence:
    listWord = sentence.split(" ")
    if "" in listWord:
        listWord.remove("")
    if len(listWord) > maxLengthSentence:
        maxLengthSentence = len(listWord)

    listWordVector.append(listWord)
    for word in listWord:
        emotionVocab.add(word)
    

PAD_TOKEN = "<PAD>"
EOS_TOKEN = "<EOS>"
SOS_TOKEN = "<SOS>"
UNK_TOKEN = "<UNK>"
listToken = [PAD_TOKEN, EOS_TOKEN, SOS_TOKEN, UNK_TOKEN]
emotionVocabList = list(emotionVocab) + listToken
sortedVocab = sorted(emotionVocabList)

print(sortedVocab)
print(listWordVector[0:5])

print(maxLengthSentence)

['<EOS>', '<PAD>', '<SOS>', '<UNK>', 'a', 'abalone', 'abandon', 'abandoned', 'abdicated', 'abdomen', 'abe', 'abilities', 'ability', 'abject', 'able', 'abolished', 'abortion', 'abotu', 'about', 'above', 'abraham', 'abroad', 'absence', 'absent', 'absentee', 'absolute', 'absolutely', 'absoulutely', 'abstract', 'absurd', 'absurdly', 'abundant', 'abundantly', 'abuout', 'abuse', 'abused', 'abusee', 'abuser', 'abusers', 'abuses', 'abusing', 'abusive', 'abut', 'ac', 'acceleration', 'accent', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'access', 'accessing', 'accident', 'accidentally', 'accidents', 'acclimate', 'acclimation', 'accompanied', 'accomplish', 'accomplished', 'accomplishes', 'accomplishing', 'accomplishment', 'accord', 'according', 'accordingly', 'accords', 'accoring', 'account', 'accountability', 'accountable', 'accounts', 'accurate', 'accusations', 'accused', 'accuser', 'accustomed', 'achieve', 'achieving', 'acknowledge', 'acknowledging', 'acquire', 'acquiring', 

In [43]:
listSentencePad = []
for wordVector in listWordVector:
    paddedWordVector = [SOS_TOKEN] + wordVector + ([PAD_TOKEN] * (maxLengthSentence - len(wordVector))) + [EOS_TOKEN]
    listSentencePad.append(paddedWordVector)
print(listSentencePad[0])
print(len(listSentencePad[0]))

['<SOS>', 'what', 'did', 'you', 'think', 'about', 'this', 'article', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PA

In [39]:
eng_word2int = {word: i for i, word in enumerate(sortedVocab)}
eng_int2word = {i: word for word, i in eng_word2int.items()}
print(eng_word2int)

{'<EOS>': 0, '<PAD>': 1, '<SOS>': 2, '<UNK>': 3, 'a': 4, 'abalone': 5, 'abandon': 6, 'abandoned': 7, 'abdicated': 8, 'abdomen': 9, 'abe': 10, 'abilities': 11, 'ability': 12, 'abject': 13, 'able': 14, 'abolished': 15, 'abortion': 16, 'abotu': 17, 'about': 18, 'above': 19, 'abraham': 20, 'abroad': 21, 'absence': 22, 'absent': 23, 'absentee': 24, 'absolute': 25, 'absolutely': 26, 'absoulutely': 27, 'abstract': 28, 'absurd': 29, 'absurdly': 30, 'abundant': 31, 'abundantly': 32, 'abuout': 33, 'abuse': 34, 'abused': 35, 'abusee': 36, 'abuser': 37, 'abusers': 38, 'abuses': 39, 'abusing': 40, 'abusive': 41, 'abut': 42, 'ac': 43, 'acceleration': 44, 'accent': 45, 'accept': 46, 'acceptable': 47, 'acceptance': 48, 'accepted': 49, 'accepting': 50, 'access': 51, 'accessing': 52, 'accident': 53, 'accidentally': 54, 'accidents': 55, 'acclimate': 56, 'acclimation': 57, 'accompanied': 58, 'accomplish': 59, 'accomplished': 60, 'accomplishes': 61, 'accomplishing': 62, 'accomplishment': 63, 'accord': 64, 

In [74]:
import numpy as np
wordTensor = []
for listSentence in listSentencePad:
    numericArray = []
    for word in listSentence:
        numericArray.append(eng_word2int[word])
    wordTensor.append(np.array(numericArray))
X_train = np.array(wordTensor)
print(wordTensor[0])

[   2 9623 2335 9873 8778   18 8789  533    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    0]


In [70]:
from numpy import unique
emotionColumnIndex = df_train.columns.get_loc("Emotion")
emotionPolarityColumnIndex = df_train.columns.get_loc("EmotionalPolarity")
emphathyColumnIndex = df_train.columns.get_loc("Empathy")

Y_train_reg = []
Y_train_class = []
instance = []
for i in range(0, len(df_train)):
    instance.append(df_train.iloc[i, emotionColumnIndex])

    instance.append(df_train.iloc[i, emphathyColumnIndex])
    Y_train_reg.append(instance)
    instance = []

for i in range(0, len(df_train)):

    instance.append(df_train.iloc[i, emotionPolarityColumnIndex])

    Y_train_class.append(instance)
    instance = []

Y_train_reg = np.array(Y_train_reg)
Y_train_class = np.array(Y_train_class)
nClass = len(unique(Y_train_class))
print(nClass)
print(Y_train_reg)
print(Y_train_class)

4
[[1 1]
 [3 4]
 [4 5]
 ...
 [3 4]
 [2 1]
 [2 1]]
[[1]
 [2]
 [2]
 ...
 [2]
 [0]
 [0]]


In [71]:
embedding_dim = 300  
embedding_index = {}
with open("glove.6B.300d.txt", "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = vector


In [72]:
# 2D Matrix dimension of Vocab Size x 300 (using glove 6B 300 dimension)

embedding_matrix = np.zeros((len(sortedVocab), embedding_dim))

for word, idx in eng_word2int.items():

    vector = embedding_index.get(word)

    if vector is not None:
        embedding_matrix[idx] = vector

    elif word == PAD_TOKEN or word == SOS_TOKEN or word == EOS_TOKEN:
        embedding_matrix[idx] = np.zeros(shape=(embedding_dim,))

    elif word == UNK_TOKEN :
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

In [75]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import LSTM, Dense
tf.random.set_seed(42)

inputs = Input(shape=(len(wordTensor[0]),))

embedding_layer = Embedding(
    input_dim = len(sortedVocab),                
    output_dim = embedding_dim,             
    embeddings_initializer = Constant(embedding_matrix),  
    input_length = len(wordTensor[0]),             
    trainable=False                       
)(inputs)

lstmLayers = LSTM(30, activation='tanh')(embedding_layer)

regression_output = Dense(2, name='regression')(lstmLayers)

classification_output = Dense(nClass, activation = 'softmax', name='classification')(lstmLayers)

modelNLP = Model(inputs = inputs, outputs=[regression_output, classification_output])

modelNLP.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'regression': 'mse',
        'classification': 'categorical_crossentropy'
    },
    metrics={
        'regression': ['mse'],
        'classification': ['accuracy']
    }
)

plot_model(modelNLP, to_file='model.png', show_shapes=True)

modelNLP.fit(
    X_train, [Y_train_reg, to_categorical(Y_train_class, num_classes = nClass)],
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.
Epoch 1/50
278/278 ━━━━━━━━━━━━━━━━━━━━ 33s 95ms/step - classification_accuracy: 0.4342 - classification_loss: 1.0603 - loss: 2.2430 - regression_loss: 1.1809 - regression_mse: 1.1825 - val_classification_accuracy: 0.4504 - val_classification_loss: 0.9734 - val_loss: 1.8279 - val_regression_loss: 0.8549 - val_regression_mse: 0.8567
Epoch 2/50
278/278 ━━━━━━━━━━━━━━━━━━━━ 26s 92ms/step - classification_accuracy: 0.4381 - classification_loss: 0.9975 - loss: 1.8018 - regression_loss: 0.8037 - regression_mse: 0.8042 - val_classification_accuracy: 0.4504 - val_classification_loss: 0.9696 - val_loss: 1.8225 - val_regression_loss: 0.8533 - val_regression_mse: 0.8551
Epoch 3/50
 74/278 ━━━━━━━━━━━━━━━━━━━━ 18s 90ms/step - classification_accuracy: 0.4353 - classification_loss: 1.0011 - loss: 1.7844 - regression_loss: 0.7833 - regression_mse: 0.7833

KeyboardInterrupt: 

In [7]:
import json
import numpy as np

def processOutput(df):
    emotionColumnIndex = df.columns.get_loc("Emotion")
    emotionPolarityColumnIndex = df.columns.get_loc("EmotionalPolarity")
    emphathyColumnIndex = df.columns.get_loc("Empathy")

    Y_reg = []
    Y_class = []
    instance = []
    df = df[0:100]

    for i in range(0, len(df)):
        instance.append(df.iloc[i, emotionColumnIndex])
        instance.append(df.iloc[i, emphathyColumnIndex])
        Y_reg.append(instance)
        instance = []

    for i in range(0, len(df)):
        instance.append(df.iloc[i, emotionPolarityColumnIndex])
        Y_class.append(instance)
        instance = []

    return np.array(Y_reg), np.array(Y_class)

In [8]:
import numpy as np

df_test = pd.read_csv("trac2_CONVT_dev.csv")
df_test["EmotionalPolarity"] = np.ceil(df_test['EmotionalPolarity'])

Y_test_reg, Y_test_class = processOutput(df_test)